In [10]:
!pip install langchain

In [11]:
!pip install langchain-groq

In [12]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-70b-versatile"
)

response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [13]:
!pip install chromadb

In [14]:
import chromadb

client = chromadb.Client()
collection = client.create_collection(name="my_collection")

UniqueConstraintError: Collection my_collection already exists

In [15]:
collection.add(
    documents=[
        "This document is about New York",
        "This document is about Delhi"
    ],
    ids = ['id1', 'id2']
)

/Users/duviniranaweera/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 1


In [16]:
all_docs = collection.get()
all_docs

{'ids': ['id1', 'id2'],
 'embeddings': None,
 'documents': ['This document is about New York',
  'This document is about Delhi'],
 'uris': None,
 'data': None,
 'metadatas': [None, None],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [17]:
documents = collection.get(ids=["id1"])
documents

{'ids': ['id1'],
 'embeddings': None,
 'documents': ['This document is about New York'],
 'uris': None,
 'data': None,
 'metadatas': [None],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [18]:
results = collection.query(
    query_texts=['Query is about big apple'],
    n_results=2
)
results

{'ids': [['id1', 'id2']],
 'embeddings': None,
 'documents': [['This document is about New York',
   'This document is about Delhi']],
 'uris': None,
 'data': None,
 'metadatas': [[None, None]],
 'distances': [[1.6312263011932373, 1.8675483465194702]],
 'included': [<IncludeEnum.distances: 'distances'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [8]:
collection.delete(ids=all_docs['ids'])
collection.get()

ValueError: Expected IDs to be a non-empty list, got 0 IDs in delete.

In [9]:
	
collection.add(
    documents=[
        "This document is about New York",
        "This document is about Delhi"
    ],
    ids=["id3", "id4"],
    metadatas=[
        {"url": "https://en.wikipedia.org/wiki/New_York_City"},
        {"url": "https://en.wikipedia.org/wiki/New_Delhi"}
    ]
)

/Users/duviniranaweera/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz:  


ReadTimeout: The read operation timed out in add.

In [19]:
results = collection.query(
    query_texts=["Query is about Chhole Bhature"],
    n_results=2
)
results

{'ids': [['id2', 'id1']],
 'embeddings': None,
 'documents': [['This document is about Delhi',
   'This document is about New York']],
 'uris': None,
 'data': None,
 'metadatas': [[None, None]],
 'distances': [[1.558892011642456, 1.8115087747573853]],
 'included': [<IncludeEnum.distances: 'distances'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [21]:
!pip install langchain-community

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 459.5 kB/s eta 0:00:00a 0:00:01


In [26]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-43537")
page_data = loader.load().pop().page_content
print(page_data)

Apply for Software Engineer II, ITC

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologySoftware Engineer II, ITCKarnataka, IndiaBecome a Part of the NIKE, Inc. TeamNIKE, Inc. doe

In [29]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [30]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Software Engineer II',
 'experience': '3-6 years',
 'skills': ['Fullstack development',
  'Front-end technologies (React)',
  'Backend development (Node.js OR Java)',
  'Microservice architecture',
  'RESTful APIs',
  'GraphQL services',
  'Cloud environments (AWS, Azure, or GCP)',
  'Test-Driven Development (TDD)',
  'JUnit, Jest, or Mocha',
  'Relational and NoSQL databases (e.g., MySQL, MongoDB, DynamoDB)',
  'CI/CD pipelines',
  'Version control',
  'Agile methodologies',
  'UI/UX best practices',
  'Performance optimization',
  'Responsive design principles'],
 'description': 'As a Software Engineer 2 - Fullstack, you will be part of a dynamic team driving the creation of both backend and frontend solutions, delivering powerful, high-performance applications across digital and in-store platforms. You will be passionate about creating seamless user experiences, efficient backend processes, and quality code.'}

In [31]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [32]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context leak detected, msgtracer returned -1
Context le

In [39]:
links = collection.query(query_texts=['skills'], n_results=2).get('metadatas', [])
links


[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}]]

In [38]:
job = json_res
job['skills']

['Fullstack development',
 'Front-end technologies (React)',
 'Backend development (Node.js OR Java)',
 'Microservice architecture',
 'RESTful APIs',
 'GraphQL services',
 'Cloud environments (AWS, Azure, or GCP)',
 'Test-Driven Development (TDD)',
 'JUnit, Jest, or Mocha',
 'Relational and NoSQL databases (e.g., MySQL, MongoDB, DynamoDB)',
 'CI/CD pipelines',
 'Version control',
 'Agile methodologies',
 'UI/UX best practices',
 'Performance optimization',
 'Responsive design principles']

In [40]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Fullstack Development Services for Seamless User Experiences

Dear Hiring Manager,

I came across the job description for a Software Engineer II - Fullstack at your esteemed organization, and I am excited to introduce AtliQ, an AI & Software Consulting company that can help you achieve your goals. With our expertise in fullstack development, we can drive the creation of both backend and frontend solutions, delivering high-performance applications that meet your requirements.

At AtliQ, we have a proven track record of empowering enterprises with tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency. Our team of experts is well-versed in the latest technologies, including React, Node.js, Java, Microservice architecture, RESTful APIs, GraphQL services, and Cloud environments (AWS, Azure, or GCP). We also follow best practices in Test-Driven Development (TDD), using JUnit, Jest, or Mocha, and have experience wit